In [ ]:
import math
import pandas as pd
from datetime import datetime

In [ ]:
user_ids = [1, 2, 4, 5, 6, 8, 10, 11, 12, 13, 15, 16, 18, 19, 20, 21, 23, 25, 26, 27, 28, 29, 31, 32, 33, 34, 37, 38, 39, 40, 44, 45, 46, 47, 48]
len(user_ids)

35

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# FPOGX: The X-coordinate of the fixation POG, as a percentage of the screen width (0 to 1)
    # Average of right & left eye data
# FPOGY: The Y-coordinate of the fixation POG, as a percentage of the screen width (0 to 1)
# FPOGS: The starting time of the fixation POG in seconds since the system initialzation or calibration 
# FPOGD: The durtaion of the fixation POG in seconds 

In [ ]:
full_screen_width = 1920
full_screen_height = 1080

ai_start_cx = 90/full_screen_width 
ai_end_cx = 555/full_screen_width 

task_start_cx = 770/full_screen_width 
task_end_cx = 1255/full_screen_width 

user_start_cx = 1385/full_screen_width 
user_end_cx = 1845/full_screen_width 

# start_cy = 275/full_screen_height
# end_cy = 685/full_screen_height

start_cy = 0/full_screen_height
end_cy = 1080/full_screen_height

In [ ]:
db = '/content/drive/MyDrive/reliance/data_analysis_files+code/'

# Response Data
user = pd.read_excel(db + "user.xlsx") 
demographic = pd.read_excel(db + "demographic.xlsx") 
practice = pd.read_excel(db + "practice.xlsx")

main1 = pd.read_excel(db + "main1.xlsx")
main2 = pd.read_excel(db + "main2.xlsx")
main3 = pd.read_excel(db + "main3.xlsx")
main = main1.append(main2)
main = main.append(main3)

time1 = pd.read_excel(db + "time1.xlsx")
time2 = pd.read_excel(db + "time2.xlsx")
time3 = pd.read_excel(db + "time3.xlsx")
time = time1.append(time2)
time = time.append(time3)

# Survey Data
practice_survey_one = pd.read_excel(db + 'practice_survey_one.xlsx')
practice_survey_two = pd.read_excel(db + 'practice_survey_two.xlsx')
practice_survey_three = pd.read_excel(db + 'practice_survey_three.xlsx')
final_survey = pd.read_excel(db + 'final_survey.xlsx')

middle_survey1 = pd.read_excel(db + 'middle_survey1.xlsx')
middle_survey2 = pd.read_excel(db + 'middle_survey2.xlsx')
middle_survey3 = pd.read_excel(db + 'middle_survey3.xlsx')
middle_survey = middle_survey1.append(middle_survey2)
middle_survey = middle_survey.append(middle_survey3)

In [ ]:
user.loc[user['User id'].isin(user_ids)]['Acc'].value_counts()

0    14
2    11
1    10
Name: Acc, dtype: int64

In [ ]:
user_acc = user[['User id', 'Acc']].groupby('User id', as_index=False).first()

In [ ]:
ending_survey = middle_survey.loc[middle_survey['Round'] == 10]
ending_survey = ending_survey.rename({'Q1': 'survey_reliance'}, axis=1)
ending_survey = ending_survey.rename({'Q2': 'survey_trust'}, axis=1)
ending_survey = ending_survey.rename({'Q3': 'survey_confidence'}, axis=1)
ending_survey = ending_survey.rename({'Q4': 'survey_agreement'}, axis=1)

middle_survey = middle_survey.loc[middle_survey['Round'] != 10]
middle_survey = middle_survey.rename({'Q2': 'survey_reliance'}, axis=1)
middle_survey = middle_survey.rename({'Q3': 'survey_trust'}, axis=1)
middle_survey = middle_survey.rename({'Q4': 'survey_confidence'}, axis=1)
middle_survey = middle_survey.rename({'Q1': 'survey_agreement'}, axis=1)

middle_survey = middle_survey.append(ending_survey)

In [ ]:
all_rounds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
user16_rounds = [1, 2, 3, 4, 5, 6, 8, 9, 10]
user28_rounds = [1, 2, 3, 4, 5, 8, 9, 10]

In [ ]:
# Create Dataframe 
user_df = pd.DataFrame(columns=['User id', 'round', 'total_duration_on_ai', 'total_duration_on_task', 'total_duration_on_user'], dtype=object)

fixation_df = pd.DataFrame(columns=['User id', 'round', 'FPOGID', 'FPOGS', 'FPOGX', 'FPOGY', 'FPOGD', 'location'], dtype=object)

shift_df = pd.DataFrame(columns=['User id', 'round', 'FPOGS', 'shift', 'orientation', 'total_duration'], dtype=object)

# Gaze Data
for user_id in user_ids:
    gaze = db + 'user-data'
    
    print('User:' + str(user_id))
    
    # User ID 
    
    all_gaze_filename = gaze + '/User ' + str(user_id) + '_all_gaze.csv'
    fixation_filename = gaze + '/User ' + str(user_id) + '_fixations.csv'

    all_gaze = pd.read_csv(all_gaze_filename, sep=',')
    fixation = pd.read_csv(fixation_filename, sep=',')
    
    # Recording Start Time
    gaze_start_time = fixation.columns[3][5:24]
    gaze_start_time_obj = datetime.strptime(gaze_start_time, '%Y/%m/%d %H:%M:%S')
    
    if user_id == 16: 
        roundslist = user16_rounds
    elif user_id == 28: 
        roundslist = user28_rounds
    else: 
        roundslist = all_rounds
    
    # Iterate through all trials 
    for i in roundslist:
        roundID = str(i) 

        trial_start_time = time.loc[(time["User id"] == user_id) & (time['Round'] == roundID)]['Start Time'].values.tolist()[0][4:24]
        trial_start_time_obj = datetime.strptime(trial_start_time, '%b %d %Y %H:%M:%S')
    
        trial_start = trial_start_time_obj - gaze_start_time_obj
        trial_start_secs = trial_start.seconds
    
        trial_end_time = time.loc[(time["User id"] == user_id) & (time['Round'] == roundID)]['End Time'].values.tolist()[0][4:24]
        trial_end_time_obj = datetime.strptime(trial_end_time, '%b %d %Y %H:%M:%S')

        trial_end = trial_end_time_obj - gaze_start_time_obj
        trial_end_secs = trial_end.seconds
        
        trial_gaze = all_gaze.loc[(all_gaze['FPOGS'] > trial_start_secs) & (all_gaze['FPOGS'] < trial_end_secs)]
        trial_fixation = trial_gaze.drop_duplicates(subset = ['FPOGID'], keep='last')[['FPOGID', 'FPOGS', 'FPOGX', 'FPOGY', 'FPOGD']]

        trial_fixation["User id"] = user_id
        trial_fixation['round'] = roundID

        duration_on_ai = trial_fixation.loc[(trial_fixation['FPOGX'] > ai_start_cx) & 
                                            (trial_fixation['FPOGX'] < ai_end_cx) & 
                                            (trial_fixation['FPOGY'] > start_cy) &
                                            (trial_fixation['FPOGY'] < end_cy)]

        trial_fixation.loc[(trial_fixation['FPOGX'] > ai_start_cx) & 
                           (trial_fixation['FPOGX'] < ai_end_cx) & 
                           (trial_fixation['FPOGY'] > start_cy) &
                           (trial_fixation['FPOGY'] < end_cy), "location"] = "AI"

        duration_on_task = trial_fixation.loc[(trial_fixation['FPOGX'] > task_start_cx) & 
                                              (trial_fixation['FPOGX'] < task_end_cx) & 
                                              (trial_fixation['FPOGY'] > start_cy) &
                                              (trial_fixation['FPOGY'] < end_cy)]

        trial_fixation.loc[(trial_fixation['FPOGX'] > task_start_cx) & 
                           (trial_fixation['FPOGX'] < task_end_cx) & 
                           (trial_fixation['FPOGY'] > start_cy) &
                           (trial_fixation['FPOGY'] < end_cy), "location"] = "Task"

        duration_on_user = trial_fixation.loc[(trial_fixation['FPOGX'] > user_start_cx) & 
                                             (trial_fixation['FPOGX'] < user_end_cx) & 
                                             (trial_fixation['FPOGY'] > start_cy) &
                                             (trial_fixation['FPOGY'] < end_cy)]

        trial_fixation.loc[(trial_fixation['FPOGX'] > user_start_cx) & 
                           (trial_fixation['FPOGX'] < user_end_cx) & 
                           (trial_fixation['FPOGY'] > start_cy) &
                           (trial_fixation['FPOGY'] < end_cy), "location"] = "User"

        total_duration_on_ai = sum(duration_on_ai['FPOGD'])
        total_duration_on_task = sum(duration_on_task['FPOGD'])
        total_duration_on_user = sum(duration_on_user['FPOGD'])

        total_duration = trial_end_secs - trial_start_secs
        trial_fixation['total_duration'] = total_duration

        percent_duration_on_ai = total_duration_on_ai / total_duration
        percent_duration_on_task = total_duration_on_task / total_duration
        percent_duration_on_user = total_duration_on_user / total_duration

        # Before or After AI suggestion is shown 
        show_ai = main.loc[(main["User id"] == user_id) & (main['Round'] == int(roundID)),"Show AItime"].values[0]
        show_ai_time = trial_start_secs + show_ai
        
        trial_fixation['fixation_start_time'] = trial_fixation['FPOGS'] - trial_start_secs
        trial_fixation['orientation'] = main.loc[(main["User id"] == user_id) & (main['Round'] == int(roundID)),"Orientation"].values[0]
        
        trial_fixation['show_ai_sec'] = show_ai
        trial_fixation['show_ai_time'] = show_ai_time
        
        trial_fixation['after_show_ai'] = False
        trial_fixation.loc[trial_fixation['FPOGS'] >= show_ai_time, "after_show_ai"] = True
       
        task_to_user = 0
        task_to_ai = 0
        user_to_task = 0
        user_to_ai = 0
        ai_to_user = 0
        ai_to_task = 0
        
        trial_fixation['drop'] = False
        cur_row = 0
        
        # Combine fixations in the same area
        for i in range(len(trial_fixation)-1):

            trial_orientation = main.loc[(main["User id"] == user_id) & (main['Round'] == int(roundID)),"Orientation"].values[0]

            if trial_fixation.iloc[cur_row]['location'] == "User":
                if trial_fixation.iloc[i+1]['location'] == "Task":
                    cur_row = i+1
                    shift_df = shift_df.append({'User id':user_id, 'round':roundID, 'orientation':trial_orientation,
                                                'FPOGS':trial_fixation.iloc[i+1]['FPOGS']- trial_start_secs, 'shift':'User2Task','total_duration':total_duration}, ignore_index=True)
                elif trial_fixation.iloc[i+1]['location'] == "AI":
                    cur_row = i+1
                    shift_df = shift_df.append({'User id':user_id, 'round':roundID, 'orientation':trial_orientation,
                                                'FPOGS':trial_fixation.iloc[i+1]['FPOGS']- trial_start_secs, 'shift':'User2AI','total_duration':total_duration}, ignore_index=True)
                elif trial_fixation.iloc[i+1]['location'] == "User":
                    trial_fixation.iloc[cur_row]['FPOGD'] = trial_fixation.iloc[cur_row]['FPOGD'] + trial_fixation.iloc[i+1]['FPOGS']
                    ID = trial_fixation.iloc[i+1]['FPOGID']
                    trial_fixation.loc[trial_fixation['FPOGID'] == ID,'drop'] = True
                else: 
                    cur_row = i+1
            elif trial_fixation.iloc[cur_row]['location'] == "Task":
                if trial_fixation.iloc[i+1]['location'] == "User":
                    cur_row = i+1
                    shift_df = shift_df.append({'User id':user_id, 'round':roundID, 'orientation':trial_orientation,
                                                'FPOGS':trial_fixation.iloc[i+1]['FPOGS']- trial_start_secs, 'shift':'Task2User', 'total_duration':total_duration}, ignore_index=True)
                elif trial_fixation.iloc[i+1]['location'] == "AI":
                    cur_row = i+1
                    shift_df = shift_df.append({'User id': user_id, 'round': roundID, 'orientation':trial_orientation,
                                                'FPOGS':trial_fixation.iloc[i+1]['FPOGS']- trial_start_secs, 'shift':'Task2AI', 'total_duration':total_duration}, ignore_index=True)
                elif trial_fixation.iloc[i+1]['location'] == "Task":
                    trial_fixation.iloc[cur_row]['FPOGD'] = trial_fixation.iloc[cur_row]['FPOGD'] + trial_fixation.iloc[i+1]['FPOGS']
                    ID = trial_fixation.iloc[i+1]['FPOGID']
                    trial_fixation.loc[trial_fixation['FPOGID'] == ID,'drop'] = True
                else: 
                    cur_row = i+1
            elif trial_fixation.iloc[cur_row]['location'] == "AI":
                if trial_fixation.iloc[i+1]['location'] == "User":
                    cur_row = i+1
                    shift_df = shift_df.append({'User id': user_id, 'round': roundID, 'orientation':trial_orientation,
                                                'FPOGS':trial_fixation.iloc[i+1]['FPOGS']- trial_start_secs, 'shift':'AI2User', 'total_duration':total_duration}, ignore_index=True)
                elif trial_fixation.iloc[i+1]['location'] == "Task":
                    cur_row = i+1
                    shift_df = shift_df.append({'User id': user_id, 'round':roundID, 'orientation':trial_orientation,
                                                'FPOGS':trial_fixation.iloc[i+1]['FPOGS']- trial_start_secs, 'shift':'AI2Task', 'total_duration':total_duration}, ignore_index=True)
                elif trial_fixation.iloc[i+1]['location'] == "AI":
                    trial_fixation.iloc[cur_row]['FPOGD'] = trial_fixation.iloc[cur_row]['FPOGD'] + trial_fixation.iloc[i+1]['FPOGS']
                    ID = trial_fixation.iloc[i+1]['FPOGID']
                    trial_fixation.loc[trial_fixation['FPOGID'] == ID,'drop'] = True
                else: 
                    cur_row = i+1
            else: 
                cur_row = i+1
            
        trial_fixation = trial_fixation.loc[trial_fixation['drop'] == False]
        fixation_df = fixation_df.append(trial_fixation)

User:1


/usr/local/lib/python3.8/dist-packages/pandas/core/series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cacher_needs_updating = self._check_is_chained_assignment_possible()


User:2
User:4
User:5
User:6
User:8
User:10
User:11
User:12
User:13
User:15
User:16
User:18
User:19
User:20
User:21
User:23
User:25
User:26
User:27


<ipython-input-11-b93b4b11fd36>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_fixation["User id"] = user_id
<ipython-input-11-b93b4b11fd36>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_fixation['round'] = roundID
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

User:28
User:29
User:31


<ipython-input-11-b93b4b11fd36>:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_fixation['show_ai_sec'] = show_ai


User:32
User:33
User:34
User:37
User:38
User:39
User:40


<ipython-input-11-b93b4b11fd36>:104: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_fixation['show_ai_time'] = show_ai_time
<ipython-input-11-b93b4b11fd36>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_fixation['after_show_ai'] = False
<ipython-input-11-b93b4b11fd36>:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

User:44
User:45
User:46
User:47
User:48


In [ ]:
# Fixation Dataframe
percent_buckets = 0.9

fixation_after = fixation_df.loc[(fixation_df['fixation_start_time'] > fixation_df['show_ai_sec'])]
fixation_after['fixation_start_time_diff_show_ai_time'] = fixation_after['fixation_start_time'] - fixation_after['show_ai_sec']
fixation_after['after_ai_appear_time'] =  fixation_after['total_duration'] - fixation_after['show_ai_sec']
fixation_after['bucket'] = fixation_after.apply(lambda x: math.floor(x['fixation_start_time_diff_show_ai_time'] / (x['after_ai_appear_time'] * percent_buckets)), axis=1)

fixation_after = fixation_after.merge(user_acc, left_on=["User id"], right_on=["User id"])

fixation_after.to_csv(db+'fixation_bucket_{}.csv'.format(percent_buckets), index=False)  

<ipython-input-12-98548bcb8048>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixation_after['fixation_start_time_diff_show_ai_time'] = fixation_after['fixation_start_time'] - fixation_after['show_ai_sec']
<ipython-input-12-98548bcb8048>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixation_after['after_ai_appear_time'] =  fixation_after['total_duration'] - fixation_after['show_ai_sec']
<ipython-input-12-98548bcb8048>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [ ]:
fixation_total_bucket = fixation_after.groupby(['User id', 'round', 'bucket'], as_index=False)[['FPOGD']].sum()
fixation_total_bucket = fixation_total_bucket.rename({'FPOGD':'fixation_total_bucket'}, axis=1) 
fixation_after = fixation_after.merge(fixation_total_bucket, left_on=["User id", 'round', 'bucket'], right_on=["User id", 'round', 'bucket'] )

fixation_after['percent duration'] = fixation_after.apply(lambda x: x['FPOGD'] / x['fixation_total_bucket'], axis=1)
df_top = fixation_after.loc[fixation_after['orientation'] == 'top'].groupby(['User id', 'round', 'bucket', 'location'], as_index=False)[['percent duration']].sum()
df_top['orientation'] = 'top'
df_bottom = fixation_after.loc[fixation_after['orientation'] == 'bottom'].groupby(['User id', 'round', 'bucket', 'location'], as_index=False)[['percent duration', 'FPOGD']].sum()
df_bottom['orientation'] = 'bottom'
df = df_top.append(df_bottom)
df = df.merge(user_acc, left_on=["User id"], right_on=["User id"])

for user_id in user_ids:
    # Input zeros
    if user_id == 16: 
        roundslist = user16_rounds
    elif user_id == 28: 
        roundslist = user28_rounds
    else: 
        roundslist = all_rounds
    
    for i in roundslist:
        for b in range (0, 2):
            for loc in ['AI', 'Task', 'User']:
                orient = df.loc[(df['User id'] == user_id) & (df['round'] == str(i))]['orientation'].values[0]
                if len(df.loc[(df['orientation'] == orient) & (df['User id'] == user_id) & (df['round'] == str(i)) & (df['bucket'] == b) & (df['location'] == loc)]) == 0:
                    df = df.append({'User id': user_id, 'round':i, 'bucket':b, 'location':loc, 'percent duration':0, 'FPOGD':0, 'orientation':'top', 'Acc':df.loc[df['User id'] == user_id]['Acc'].values[0]}, ignore_index=True)

df.to_csv(db+'bucket_gaze_duration_divided_by_fixation_total_{}.csv'.format(percent_buckets), index=False)

In [ ]:
fixation_total = fixation_after.groupby(['User id', 'round', 'bucket'], as_index=False)[['FPOGD']].sum()
fixation_total = fixation_total.rename({'FPOGD':'fixation_total_bucket'}, axis=1) 
fixation_after = fixation_after.merge(fixation_total[['User id', 'round', 'bucket', 'fixation_total_bucket']], left_on=['User id','round', 'bucket'], right_on=['User id', 'round',  'bucket'])

In [ ]:
show_ai_time_df = fixation_df.groupby(['User id', 'round'], as_index=False).first()[['User id', 'round', 'show_ai_sec']].reset_index()
show_ai_time_df['round'] = show_ai_time_df['round'].astype(int).astype(str)

shift_df_merged= shift_df.merge(show_ai_time_df[['User id', 'round','show_ai_sec']], left_on=['User id', 'round'], right_on=['User id', 'round'])

shift_after = shift_df_merged.loc[shift_df_merged['FPOGS'] > shift_df_merged['show_ai_sec']]

In [ ]:
shift = shift_after.copy()
shift['count'] = 1

shift['shift_start_time_diff_show_ai_time'] = shift['FPOGS'] - shift['show_ai_sec']
shift['after_ai_appear_time'] =  shift['total_duration'] - shift['show_ai_sec']

shift['bucket'] = shift.apply(lambda x: math.floor(x['shift_start_time_diff_show_ai_time'] / (x['after_ai_appear_time'] * percent_buckets)), axis=1)

shift_bucket_total = shift.groupby(['User id', 'round', 'bucket', 'shift'], as_index=False)[['count']].sum()

shift_total = shift.groupby(['User id', 'round', 'bucket'], as_index=False)[['count']].sum()
shift_total = shift_total.rename({'count':'shift_total_bucket'}, axis=1) 

shift_appear_time = shift.groupby(['User id', 'round', 'bucket'], as_index=False)[['after_ai_appear_time']].first()

shift = shift_bucket_total.merge(shift_total[['User id', 'round', 'bucket', 'shift_total_bucket']], left_on=['User id','round', 'bucket'], right_on=['User id', 'round', 'bucket'], how='left')
shift = shift.merge(shift_appear_time, left_on=['User id','round', 'bucket'], right_on=['User id', 'round', 'bucket'], how='left')

shift['percent_shift'] = shift.apply(lambda x: x['count'] / x['shift_total_bucket'], axis=1)
shift['shift_per_sec'] = shift.apply(lambda x: x['count'] / (x['after_ai_appear_time'] * percent_buckets), axis=1)

In [ ]:
orientation_df = shift_df.groupby(['User id', 'round'], as_index=False).first()[['User id', 'round', 'orientation']]
shift = shift.merge(orientation_df, left_on=['User id','round'], right_on=['User id', 'round'], how='left')

for user_id in user_ids:
    # Input zeros
    if user_id == 16: 
        roundslist = user16_rounds
    elif user_id == 28: 
        roundslist = user28_rounds
    else: 
        roundslist = all_rounds
    
    for i in roundslist:
        for b in range (0, 2):
            for loc in ['Task2User', 'AI2User', 'User2Task', 'AI2Task', 'User2AI', 'Task2AI']:
                if len(shift.loc[(shift['User id'] == user_id) & (shift['round'] == str(i)) & (shift['bucket'] == b) & (shift['shift'] == loc)]) == 0:
                    orient = shift.loc[(shift['User id'] == user_id) & (shift['round'] == str(i))]['orientation'].values[0]
                    shift = shift.append({'User id': user_id, 'round':str(i), 'bucket':b, 'shift':loc, 'count':0, 'shift_total_bucket':0, 'percent_shift':0.0,'orientation':orient, 'shift_per_sec':0.0}, ignore_index=True)

shift = shift.merge(user[['User id', 'Acc']], left_on=["User id"], right_on=["User id"])
shift.to_csv(db+'bucket_gaze_shift_{}.csv'.format(percent_buckets), index=False) 

In [ ]:
shift

,User id,round,bucket,shift,count,shift_total_bucket,after_ai_appear_time,percent_shift,shift_per_sec,orientation,Acc
0,1,1,0,AI2Task,3,17,17.0,0.176471,0.196078,top,1
1,1,1,0,AI2User,1,17,17.0,0.058824,0.065359,top,1
2,1,1,0,Task2AI,1,17,17.0,0.058824,0.065359,top,1
3,1,1,0,Task2User,5,17,17.0,0.294118,0.326797,top,1
4,1,1,0,User2AI,3,17,17.0,0.176471,0.196078,top,1
...,...,...,...,...,...,...,...,...,...,...,...
4159,48,8,1,Task2AI,0,0,NaN,0.000000,0.000000,bottom,1
4160,48,9,0,AI2Task,0,0,NaN,0.000000,0.000000,top,1
4161,48,9,1,User2Task,0,0,NaN,0.000000,0.000000,top,1
4162,48,9,1,Task2AI,0,0,NaN,0.000000,0.000000,top,1


In [ ]:
shift.loc[shift['shift'] == 'Task2User', 'shift'] = '2User'
shift.loc[shift['shift'] == 'AI2User', 'shift'] = '2User'
shift.loc[shift['shift'] == 'User2Task', 'shift'] = '2Task'
shift.loc[shift['shift'] == 'AI2Task', 'shift'] = '2Task'
shift.loc[shift['shift'] == 'User2AI', 'shift'] = '2AI'
shift.loc[shift['shift'] == 'Task2AI', 'shift'] = '2AI'

shift_less = shift.groupby(['User id', 'round', 'bucket', 'shift'], as_index=False)[['percent_shift', 'shift_per_sec']].sum()
shift_by = shift[['User id', 'round', 'orientation', 'Acc']].groupby(['User id', 'round'], as_index=False).first()
shift_less_merged = shift_less.merge(shift_by[['User id', 'round', 'orientation', 'Acc']], left_on=["User id", 'round'], right_on=["User id", 'round'])

shift_less_merged.to_csv(db+'bucket_gaze_shift_less_{}.csv'.format(percent_buckets), index=False) 

In [ ]:
shift_less_merged

,User id,round,bucket,shift,percent_shift,shift_per_sec,orientation,Acc
0,1,1,0,2AI,0.235294,0.261438,top,1
1,1,1,0,2Task,0.411765,0.457516,top,1
2,1,1,0,2User,0.352941,0.392157,top,1
3,1,1,1,2AI,0.000000,0.000000,top,1
4,1,1,1,2Task,0.000000,0.000000,top,1
...,...,...,...,...,...,...,...,...
2077,48,9,0,2Task,0.440000,0.531401,top,1
2078,48,9,0,2User,0.480000,0.579710,top,1
2079,48,9,1,2AI,0.250000,0.048309,top,1
2080,48,9,1,2Task,0.250000,0.048309,top,1
